# Atelier sur la génétique statistique et l'épidémiologie génétique STAGE-Québec
## Thème 2 - Phénotypes moléculaires en épidémiologie génétique

Par Marc-André Legault (Université de Montréal) et Qihuang Zhang (Université McGill)

**31 juillet - 1er août 2025**

### Introduction

Ce cahier présente l'utilisation d'une deuxième approche TWAS, **FUSION**, appliquée aux mêmes statistiques sommaires GWAS que précédemment. Cette analyse comparative permet de :

1. Comprendre le fonctionnement et l'exécution de FUSION
2. Comparer les résultats avec ceux obtenus par S-PrediXcan 
3. Analyser les similitudes et différences entre ces deux méthodes TWAS

FUSION (Functional Summary-based ImputatiON) constitue une alternative méthodologique à S-PrediXcan, utilisant une approche différente pour l'imputation de l'expression génique et l'analyse d'association. Cette comparaison offre une perspective élargie des outils TWAS disponibles.

### Organisation des données

La structure de fichiers pour les données FUSION utilisées dans ce cahier est organisée comme suit :

<pre>
/data/FUSION/
├── gwas/      # Statistiques sommaires GWAS (format spécifique à FUSION)
├── ld/        # Panneaux de référence de déséquilibre de liaison (1000 Genomes EUR)
└── models/    # Modèles de pondération de l'expression génique
    ├── GTExv8_EUR_Pancreas/    # Modèles dérivés du pancréas (GTEx v8)
    └── YFS_BLOOD_RNAARR/       # Modèles dérivés du sang (Young Finns Study)
</pre>

**Contenu de chaque dossier de modèles :**

Chaque sous-dossier `models/` contient :
- Des fichiers de modèles de pondération `*.wgt.RDat` (un par chromosome)
- Un fichier `.pos` répertoriant tous les gènes disponibles avec leurs coordonnées génomiques

Cette organisation facilite l'analyse par chromosome et permet une gestion efficace des ressources computationnelles.

### Exécution de FUSION

L'analyse débute avec le premier modèle de prédiction d'expression génique, entraîné à partir de données d'expression génique dans le sang total de la cohorte **Young Finns Study (YFS)**.

Le code ci-dessous exécute FUSION chromosome par chromosome. **Note importante :** Pour des raisons de temps de calcul et de ressources, l'analyse est limitée aux deux premiers chromosomes. Pour analyser le transcriptome complet, il suffit de désactiver la limitation en commentant l'instruction `break` (ajout de `#` au début de la ligne).

*Un point de contrôle avec les résultats complets a été préparé pour économiser les ressources computationnelles lors de l'atelier.*

In [ ]:
for chrom in $(seq 1 22); do
    Rscript /workshop/software/FUSION/FUSION.assoc_test.R \
        --sumstats /workshop/data/FUSION/gwas/Mahajan.NatGen2022.DIAMANTE-EUR.ldsc_ss.FUSION_ref.tsv \
        --weights /workshop/data/FUSION/models/YFS_BLOOD_RNAARR/YFS.BLOOD.RNAARR.pos \
        --weights_dir /workshop/data/FUSION/models/YFS_BLOOD_RNAARR/ \
        --ref_ld_chr /workshop/data/FUSION/ld/1000G.EUR. \
        --chr $chrom \
        --out results/fusion_YFS_$chrom.tsv
        
    if (( $chrom > 2 )); then
        # Commentez la ligne ci-dessous pour exécuter tous les chromosomes,
        # sinon seulement les deux premiers seront utilisés.
        break
    fi
done

### Analyse comparative avec les modèles GTEx (pancréas)

L'analyse suivante utilise des modèles de pondération estimés à partir des données **GTEx v8 (pancréas)**. Ces modèles présentent une pertinence particulière pour l'étude du diabète, le pancréas étant l'organe central dans la régulation de la glycémie.

**Comparaison méthodologique :** Ces données GTEx sont identiques à celles utilisées pour créer les modèles S-PrediXcan analysés précédemment. Cette approche permet une comparaison directe des deux méthodes TWAS (FUSION vs S-PrediXcan) en utilisant les mêmes données de référence d'expression génique, isolant ainsi les différences méthodologiques entre les deux approches.

In [ ]:
for chrom in $(seq 1 22); do
    Rscript /workshop/software/FUSION/FUSION.assoc_test.R \
        --sumstats /workshop/data/FUSION/gwas/Mahajan.NatGen2022.DIAMANTE-EUR.ldsc_ss.FUSION_ref.tsv \
        --weights /workshop/data/FUSION/models/GTExv8_EUR_Pancreas/GTExv8.EUR.Pancreas.pos \
        --weights_dir /workshop/data/FUSION/models/GTExv8_EUR_Pancreas \
        --ref_ld_chr /workshop/data/FUSION/ld/1000G.EUR. \
        --chr $chrom \
        --out results/fusion_GTEx_pancreas_$chrom.tsv
        
    if (( $chrom > 2 )); then
        # Commentez la ligne ci-dessous pour exécuter tous les chromosomes,
        # sinon seulement les deux premiers seront utilisés.
        break
    fi
done

### Consolidation des résultats

Pour faciliter l'analyse downstream et la comparaison entre chromosomes, il est possible de consolider tous les résultats en un seul fichier par tissu. Le code ci-dessous est commenté mais peut être activé si nécessaire :

In [ ]:
# Fusionner les fichiers (GTEx Pancréas).
# cp results/fusion_GTEx_pancreas_1.tsv results/fusion_GTEx_pancreas.tsv
# for chrom in $(seq 2 22); do
#     cat results/fusion_GTEx_pancreas_${chrom}.tsv | sed 1d >> results/fusion_GTEx_pancreas.tsv
# done
#
# Fusionner les fichiers (YFS).
# cp results/fusion_YFS_1.tsv results/fusion_YFS.tsv
# for chrom in $(seq 2 22); do
#     cat results/fusion_YFS_${chrom}.tsv | sed 1d >> results/fusion_YFS.tsv
# done

### Structure des résultats

L'examen de la structure des résultats TWAS générés constitue l'étape suivante. Les fichiers peuvent être ouverts en double-cliquant dans le navigateur de fichiers, ou via la commande ci-dessous pour examiner les en-têtes de colonnes :

Les fichiers de sortie FUSION contiennent plusieurs colonnes informatives organisées de manière systématique :

In [ ]:
head -n 1 /workshop/local/results/fusion_GTEx_pancreas_1.tsv | tr '\t' '\n' | nl

### Interprétation des résultats FUSION

**Colonnes principales des fichiers de sortie :**

- **TWAS.Z, TWAS.P** : Score Z et valeur p pour l'association gène-trait imputée (statistiques principales TWAS)
- **MODELCV.R2, MODELCV.PV** : Performance prédictive du modèle évaluée par validation croisée (R² et valeur p)
- **Best.GWAS.Z** : Signal SNP GWAS le plus fort dans la région génique (pour évaluer la colinéarité)

**Éléments d'interprétation :**

1. **TWAS.P** constitue la statistique principale d'association gène-trait
2. **MODELCV.R2** indique la qualité prédictive du modèle (valeur élevée = fiabilité accrue)
3. **Best.GWAS.Z** permet de distinguer les signaux géniques des signaux SNP directs

L'extrait formaté ci-dessous facilite l'interprétation des résultats :

In [ ]:
awk '{print $1, $4, $5, $9, $10, $11, $19, $20}' /workshop/local/results/fusion_GTEx_pancreas_1.tsv | head

In [ ]:
wc -l ./results/fusion_GTEx_pancreas.tsv
wc -l ./results/fusion_YFS.tsv

<div class="alert alert-info">
<strong>Exercice pratique 1 : Correction multiple et significativité</strong>

Identifiez les gènes dans chaque tissu qui atteignent le seuil de significativité après correction de Bonferroni à l'échelle du tissu (α = 0,05 / nombre de gènes testés par tissu).

**Questions d'analyse :**
1. Combien de gènes significatifs sont identifiés dans chaque tissu ?
2. Quelle différence observe-t-on dans le nombre de découvertes entre les deux tissus ?
3. Cette différence reflète-t-elle la pertinence biologique des tissus pour le diabète ?

**Méthode :** Utilisez les commandes `awk` et `wc -l` pour compter le nombre total de gènes testés, puis identifiez ceux qui atteignent le seuil ajusté.

<!--
Solution:
awk 'NR == 1 || $20 <= 0.05/5765 { print $3, $19, $20 }' results/fusion_GTEx_pancreas.tsv
awk 'NR == 1 || $20 <= 0.05/4628 { print $3, $19, $20 }' results/fusion_YFS.tsv

# 119 significatifs pour GTEx pancréas
# 85 significatifs pour YFS
//-->

</div>

<div class="alert alert-info">
<strong>Exercice pratique 2 : Évaluation de la qualité des modèles</strong>

Examinez la distribution des valeurs **MODELCV.R2** dans les résultats obtenus.

**Questions d'analyse :**
1. Comment la performance prédictive (R²) du modèle influence-t-elle la confiance dans les associations TWAS identifiées ?
2. Un filtrage des résultats selon un seuil minimal de R² serait-il approprié ? Si oui, quel seuil pourrait être appliqué ?
3. Existe-t-il une corrélation entre la significativité TWAS et la qualité du modèle de prédiction ?

**Approche analytique :** Créez des histogrammes des valeurs R² et examinez la relation entre R² et significativité TWAS.
</div>

<div class="alert alert-info">
<strong>Exercice pratique 3 : Comparaison inter-tissus</strong>

Comparez les signaux géniques entre les deux tissus analysés (pancréas vs sang).

**Questions d'investigation :**
1. Certains signaux géniques présentent-ils une intensité variable selon le tissu ?
2. Quels gènes montrent des associations significatives dans les deux tissus ?
3. Comment interpréter les différences observées d'un point de vue biologique ?

**Approches analytiques :**
- Identifiez les gènes communs entre les deux analyses
- Comparez les scores Z TWAS pour les gènes partagés
- Analysez les implications biologiques des différences tissulaires

**Perspective :** Cette comparaison constitue une préparation à la comparaison entre FUSION et S-PrediXcan dans la section suivante.
</div>